# 0 Init

In [ ]:
!pip install datasets
!pip install transformers
!pip install --upgrade xlrd
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 22.8 MB/s 
     |████████████████████████████████| 212 kB 71.9 MB/s 
     |████████████████████████████████| 132 kB 62.7 MB/s 
     |████████████████████████████████| 182 kB 69.1 MB/s 
     |████████████████████████████████| 127 kB 74.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 18.4 MB/s 
     |████████████████████████████████| 7.6 MB 42.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Unins

In [ ]:
#import our model from huggingface
number_classes = 2
model_tokenizer = "hfl/chinese-bert-wwm"

if number_classes == 2:
    model_classifier = "owen198/weibo-wmmbert-2"
else:
    model_classifier = "owen198/weibo-wmmbert-6"


In [ ]:
model_classifier

'owen198/weibo-wmmbert-2'

#**柯文哲_before**

# 1. load and clean data

In [ ]:
#insert new xlsx file here
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/柯文哲_before.xlsx

File ‘柯文哲_before.xlsx’ already there; not retrieving.



In [ ]:
import pandas as pd

In [ ]:
#insert new xlsx file here
testing_pd = pd.read_excel('/content/柯文哲_before.xlsx')

testing_pd.rename(columns={'博文': 'text'},
          inplace=True, errors='raise')

testing_pd = testing_pd[['text']]

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#dirty data
testing_pd

,text
0,【参选后与柯P切割？黄珊珊：政绩息息相关 没什么好切割】台北市副市长黄珊珊明日将以无党籍平民...
1,#关注台海# 湾湾冥进党不顾台海紧张情势拼选举，柯文哲怒呛: 把战争当游戏#消灭台独##解放...
2,【柯文哲点评仇恨值高 陈时中亏：是他对我仇恨值高】台北市长柯文哲昨接受专访谈及台北市长选举「...
3,冥进党不顾台海紧张情势拼选举，柯文哲怒呛:把战争当游戏#台湾# L泉州那些事儿的微博视频 ​
4,【#民进党不顾台海紧张情势拼选举# 柯文哲呛:把战争当游戏】 L台海网的微博视频 ​
...,...
611,学店仔笑死人。现在是柯文哲要选新竹市长？？ ​
612,【台湾特稿：首次参与台湾“九合一”选举 蓝绿夹缝中 白营不投降】台湾政治版图长期由蓝绿（国民...
613,#国台办称对团团病逝感到痛心和惋惜# 看看柯文哲这德行，他们不配！? O网页链接 ​
614,【柯文哲质疑竹市有不可告人之事 林智坚脸书长文反击】民众党新竹市长候选人高虹安助理费争议延烧...


In [ ]:
#we can consider not deleting the titles. This will be up to the ICI project group to decide
#if you wish to preserve the titles. replace the corresponding block of code below with this one 

#testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True)#.replace(title_pattern_1,'',regex=True)
#testing_pd['text'] = testing_pd['text'].re place(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
##testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)
#________________________________________________________________________________________________________________________

testing_pd = testing_pd[~testing_pd['text'].str.contains('a href', regex=True)]


import re
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
http_pattern = "http\S+"
image_pattern = "<img\s[^>]*?src\s*=\s*['\"]([^'\"]*?)['\"][^>]*?>" 
#title_pattern = "[【|《*】|》]"
title_pattern_1 = "(?=【)(.*)(?<=】)"
title_pattern_2 = "(?=《)(.*)(?<=》)"
link_pattern="网页链接"
span_pattern='<span.*?>(.+?)</span>'

#replace with this block. Do some more research because it looks like some titles are still removed, like index 41
testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True).replace(title_pattern_1,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)

In [ ]:
#delete empty rows
testing_pd = testing_pd[testing_pd['text'].str.len()>2]   
testing_pd = testing_pd[['text']]
testing_pd = testing_pd.reset_index(drop=True)
testing_pd

,text
0,台北市副市长黄珊珊明日将以无党籍平民身份宣布参选台北市长，而传出市长柯文哲也有受邀出席参选记...
1,#关注台海# 湾湾冥进党不顾台海紧张情势拼选举，柯文哲怒呛: 把战争当游戏#消灭台独##解放...
2,台北市长柯文哲昨接受专访谈及台北市长选举「三脚督」，分析三人中只有民进党参选人陈时中仇恨值很...
3,冥进党不顾台海紧张情势拼选举，柯文哲怒呛:把战争当游戏#台湾# L泉州那些事儿的微博视频 ​
4,L台海网的微博视频 ​
...,...
611,学店仔笑死人。现在是柯文哲要选新竹市长？？ ​
612,台湾政治版图长期由蓝绿（国民党和民进党）瓜分，但也有异军突起的例外。八年前无党籍的医学教授柯...
613,#国台办称对团团病逝感到痛心和惋惜# 看看柯文哲这德行，他们不配！? O ​
614,民众党新竹市长候选人高虹安助理费争议延烧，台北市长柯文哲为护高虹安，质疑新竹市有何不可告人之...


## 1-2 Tokenization

In [ ]:
from datasets import Dataset, DatasetDict

weibo_dict = {
              'test' : Dataset.from_pandas(testing_pd)}

weibo_ds = DatasetDict(weibo_dict)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/config.json
Model config BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=512)

tokenized_datasets = weibo_ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
full_test_dataset = tokenized_datasets["test"]

# 2. preparing BERT


In [ ]:
from transformers import AutoModelForSequenceClassification

# available chinese models: https://github.com/ymcui/Chinese-BERT-wwm

model = AutoModelForSequenceClassification.from_pretrained(model_classifier, 
                                                           num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--owen198--weibo-wmmbert-2/snapshots/1c8b069ca4f42474887b1be5ddb45d0b190b785d/config.json
Model config BertConfig {
  "_name_or_path": "owen198/weibo-wmmbert-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "f

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    eval_dataset=full_test_dataset,
    compute_metrics=compute_metrics,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

#prediction results


In [ ]:
pred = trainer.predict(test_dataset=full_test_dataset)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 616
  Batch size = 8


In [ ]:
pred

PredictionOutput(predictions=array([[-2.3587573,  2.4713686],
       [-1.7682395,  1.7042342],
       [ 1.590708 , -2.0266585],
       ...,
       [ 2.4930842, -3.7055945],
       [ 1.9809703, -2.583697 ],
       [ 1.6942364, -2.2184827]], dtype=float32), label_ids=None, metrics={'test_runtime': 21.7484, 'test_samples_per_second': 28.324, 'test_steps_per_second': 3.54})

In [ ]:
# logits, labels = pred.predictions
np.argmax(pred.predictions, axis=-1)

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [ ]:
testing_pd['pred'] = np.argmax(pred.predictions, axis=-1)

In [ ]:
#final table
testing_pd

,text,pred
0,台北市副市长黄珊珊明日将以无党籍平民身份宣布参选台北市长，而传出市长柯文哲也有受邀出席参选记...,1
1,#关注台海# 湾湾冥进党不顾台海紧张情势拼选举，柯文哲怒呛: 把战争当游戏#消灭台独##解放...,1
2,台北市长柯文哲昨接受专访谈及台北市长选举「三脚督」，分析三人中只有民进党参选人陈时中仇恨值很...,0
3,冥进党不顾台海紧张情势拼选举，柯文哲怒呛:把战争当游戏#台湾# L泉州那些事儿的微博视频 ​,0
4,L台海网的微博视频 ​,0
...,...,...
611,学店仔笑死人。现在是柯文哲要选新竹市长？？ ​,1
612,台湾政治版图长期由蓝绿（国民党和民进党）瓜分，但也有异军突起的例外。八年前无党籍的医学教授柯...,1
613,#国台办称对团团病逝感到痛心和惋惜# 看看柯文哲这德行，他们不配！? O ​,0
614,民众党新竹市长候选人高虹安助理费争议延烧，台北市长柯文哲为护高虹安，质疑新竹市有何不可告人之...,0


In [ ]:
count0 = (testing_pd['pred'] == 0).sum()
count1 = (testing_pd['pred'] == 1).sum()
print(count0, count1)

401 215


In [ ]:
def Percentage(num1, num2):
    return num1 / (num1 + num2) * 100
    
print(Percentage(count0, count1))
print(Percentage(215, 401))

65.09740259740259
34.9025974025974


#**柯文哲_after**

# 1. load and clean data

In [ ]:
#insert new xlsx file here
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/柯文哲_after.xlsx

--2022-12-07 04:06:43--  https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/%E6%9F%AF%E6%96%87%E5%93%B2_after.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21632 (21K) [application/octet-stream]
Saving to: ‘柯文哲_after.xlsx’

柯文哲_after.xlsx   100%[===================>]  21.12K  --.-KB/s    in 0s      

2022-12-07 04:06:43 (136 MB/s) - ‘柯文哲_after.xlsx’ saved [21632/21632]



In [ ]:
import pandas as pd

In [ ]:
#insert new xlsx file here
testing_pd = pd.read_excel('/content/柯文哲_after.xlsx')

testing_pd.rename(columns={'博文': 'text'},
          inplace=True, errors='raise')

testing_pd = testing_pd[['text']]

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#dirty data
testing_pd

,text
0,公務員是服務人民不是管理人民因為老百姓繳錢給政府是叫政府來服務他不是叫政府來管他難道你會繳一...
1,訂了明天的文華東方辦開票趴，希望結果是好的，彌補我眼瞎投了柯文哲兩次的錯誤。 ​
2,還有35分鐘，我到底要不要去投呢？藍綠都討厭，想投柯文哲民眾黨，可是想想，會不會又是一個怪獸...
3,这年头整天自我感动的感性废物疯批好多。前些天团团在台北动物园安乐死了。看到熊猫粉中的脑残还真...
4,【柯建铭选前脸书道歉 高虹安：应向全台、全新竹市民道歉】立委高金素梅25日中午到竹市与民众党...
5,柯文哲完成投票#台湾县市长选举# ​
6,【黄珊珊市长落马 与柯革命情感祝续拚2024】无党籍黄珊珊于台北市长选战落马，市政专家形象终...
7,柯文哲可能回台大当医生去了。搞不好2年后出来竞争大统领之位。 ​
8,【国民党大胜 学者分析：朱立伦势必进取2024】#蔡英文辞去民进党主席职务# 新竹市长候选人...
9,柯文哲哭了...... ​


In [ ]:
#we can consider not deleting the titles. This will be up to the ICI project group to decide
#if you wish to preserve the titles. replace the corresponding block of code below with this one 

#testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True)#.replace(title_pattern_1,'',regex=True)
#testing_pd['text'] = testing_pd['text'].re place(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
##testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)
#________________________________________________________________________________________________________________________

testing_pd = testing_pd[~testing_pd['text'].str.contains('a href', regex=True)]


import re
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
http_pattern = "http\S+"
image_pattern = "<img\s[^>]*?src\s*=\s*['\"]([^'\"]*?)['\"][^>]*?>" 
#title_pattern = "[【|《*】|》]"
title_pattern_1 = "(?=【)(.*)(?<=】)"
title_pattern_2 = "(?=《)(.*)(?<=》)"
link_pattern="网页链接"
span_pattern='<span.*?>(.+?)</span>'

#replace with this block. Do some more research because it looks like some titles are still removed, like index 41
testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True).replace(title_pattern_1,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)

In [ ]:
#delete empty rows
testing_pd = testing_pd[testing_pd['text'].str.len()>2]   
testing_pd = testing_pd[['text']]
testing_pd = testing_pd.reset_index(drop=True)
testing_pd

,text
0,公務員是服務人民不是管理人民因為老百姓繳錢給政府是叫政府來服務他不是叫政府來管他難道你會繳一...
1,訂了明天的文華東方辦開票趴，希望結果是好的，彌補我眼瞎投了柯文哲兩次的錯誤。 ​
2,還有35分鐘，我到底要不要去投呢？藍綠都討厭，想投柯文哲民眾黨，可是想想，會不會又是一個怪獸...
3,这年头整天自我感动的感性废物疯批好多。前些天团团在台北动物园安乐死了。看到熊猫粉中的脑残还真...
4,立委高金素梅25日中午到竹市与民众党新竹市长候选人高虹安合体车扫，针对立委柯建铭在脸书对柯文...
5,柯文哲完成投票#台湾县市长选举# ​
6,无党籍黄珊珊于台北市长选战落马，市政专家形象终不敌蓝绿框架。由于选战中民众党大力支持黄珊珊，...
7,柯文哲可能回台大当医生去了。搞不好2年后出来竞争大统领之位。 ​
8,#蔡英文辞去民进党主席职务# 新竹市长候选人高虹安当选新竹市长，文化大学广告学系专任教授兼系...
9,柯文哲哭了...... ​


## 1-2 Tokenization

In [ ]:
from datasets import Dataset, DatasetDict

weibo_dict = {
              'test' : Dataset.from_pandas(testing_pd)}

weibo_ds = DatasetDict(weibo_dict)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/config.json
Model config BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=512)

tokenized_datasets = weibo_ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
full_test_dataset = tokenized_datasets["test"]

# 2. preparing BERT


In [ ]:
from transformers import AutoModelForSequenceClassification

# available chinese models: https://github.com/ymcui/Chinese-BERT-wwm

model = AutoModelForSequenceClassification.from_pretrained(model_classifier, 
                                                           num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--owen198--weibo-wmmbert-2/snapshots/1c8b069ca4f42474887b1be5ddb45d0b190b785d/config.json
Model config BertConfig {
  "_name_or_path": "owen198/weibo-wmmbert-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "f

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    eval_dataset=full_test_dataset,
    compute_metrics=compute_metrics,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

#prediction results


In [ ]:
pred = trainer.predict(test_dataset=full_test_dataset)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 48
  Batch size = 8


In [ ]:
pred

PredictionOutput(predictions=array([[ 2.3000848, -3.0898929],
       [ 2.029702 , -2.6360621],
       [ 2.4181595, -3.5218954],
       [ 1.7067909, -2.2391717],
       [ 2.2244596, -2.8827317],
       [-2.290113 ,  2.414915 ],
       [-2.3027043,  2.425799 ],
       [ 2.270879 , -2.9862928],
       [-2.5620394,  2.609875 ],
       [ 2.4219368, -3.5322108],
       [-2.1363454,  2.2541666],
       [-1.8009064,  1.753889 ],
       [-1.4535952,  1.3877127],
       [ 1.6195257, -2.0823457],
       [-1.8636042,  1.8563293],
       [-1.7485776,  1.676235 ],
       [-2.5131073,  2.5794485],
       [-2.3281455,  2.4470212],
       [ 1.7055099, -2.236999 ],
       [ 2.4874508, -3.6936102],
       [ 1.438756 , -1.6031299],
       [-2.181179 ,  2.307069 ],
       [-1.0689741,  1.1522903],
       [-1.852633 ,  1.8381356],
       [-2.2559137,  2.3838878],
       [-2.5523407,  2.603951 ],
       [ 1.672121 , -2.1804464],
       [-2.110547 ,  2.2214897],
       [-2.0864017,  2.189674 ],
       [-2.105

In [ ]:
# logits, labels = pred.predictions
np.argmax(pred.predictions, axis=-1)

array([0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1])

In [ ]:
testing_pd['pred'] = np.argmax(pred.predictions, axis=-1)

In [ ]:
#final table
testing_pd

,text,pred
0,公務員是服務人民不是管理人民因為老百姓繳錢給政府是叫政府來服務他不是叫政府來管他難道你會繳一...,0
1,訂了明天的文華東方辦開票趴，希望結果是好的，彌補我眼瞎投了柯文哲兩次的錯誤。 ​,0
2,還有35分鐘，我到底要不要去投呢？藍綠都討厭，想投柯文哲民眾黨，可是想想，會不會又是一個怪獸...,0
3,这年头整天自我感动的感性废物疯批好多。前些天团团在台北动物园安乐死了。看到熊猫粉中的脑残还真...,0
4,立委高金素梅25日中午到竹市与民众党新竹市长候选人高虹安合体车扫，针对立委柯建铭在脸书对柯文...,0
5,柯文哲完成投票#台湾县市长选举# ​,1
6,无党籍黄珊珊于台北市长选战落马，市政专家形象终不敌蓝绿框架。由于选战中民众党大力支持黄珊珊，...,1
7,柯文哲可能回台大当医生去了。搞不好2年后出来竞争大统领之位。 ​,0
8,#蔡英文辞去民进党主席职务# 新竹市长候选人高虹安当选新竹市长，文化大学广告学系专任教授兼系...,1
9,柯文哲哭了...... ​,0


In [ ]:
count0 = (testing_pd['pred'] == 0).sum()
count1 = (testing_pd['pred'] == 1).sum()
print(count0, count1)

16 32


In [ ]:
def Percentage(num1, num2):
    return num1 / (num1 + num2) * 100
    
print(Percentage(count0, count1))
print(Percentage(count1, count0))

33.33333333333333
66.66666666666666


#**蔡英文_before**

# 1. load and clean data

In [ ]:
#insert new xlsx file here
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/蔡英文_before.xlsx

--2022-12-07 04:21:42--  https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/%E8%94%A1%E8%8B%B1%E6%96%87_before.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62145 (61K) [application/octet-stream]
Saving to: ‘蔡英文_before.xlsx’

蔡英文_before.xlsx  100%[===================>]  60.69K  --.-KB/s    in 0.001s  

2022-12-07 04:21:42 (52.0 MB/s) - ‘蔡英文_before.xlsx’ saved [62145/62145]



In [ ]:
import pandas as pd

In [ ]:
#insert new xlsx file here
testing_pd = pd.read_excel('/content/蔡英文_before.xlsx')

testing_pd.rename(columns={'博文': 'text'},
          inplace=True, errors='raise')

testing_pd = testing_pd[['text']]

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#dirty data
testing_pd

,text
0,声光伴读《蔡英文撑手Joke只系吹水？香港移民「台湾梦」破碎》作者：只抽侠移居台湾的YouT...
1,粤读博览《台湾特区大势所趋！蔡英文太阿Q随时仆直！》作者：说书人当地有记者问拜登：「乌克兰现...
2,【台媒：綠營“大佬”相繼“狂倒戈”，前“立委”也出面批評民進黨】據台灣中時新聞網16日報道，...
3,【台积电创办人张忠谋第六次代表台湾出席APEC】蔡英文20日宣布由现年91岁的台积电创办人张...
4,#湾湾发电姬# 【台北没戏了！#蔡英文心腹捅出大娄子# ，疫苗黑幕败露成民进党噩梦】陈时中的...
...,...
181,【悼念拜登的狗 对“团团”不闻不问 #马英九轰蔡英文“太过分”#】#赠台大熊猫“团团”不...
182,【今日網圖】「小英」技窮？ //蔡英文獨孤一味搞政治，台灣民眾仲咁蠢想繼續捱窮咩？// ​
183,【Residentes de Taiwan lloran la partida de Tua...
184,【熊猫旅台14年，不如拜登的狗？】大陆赠台大熊猫“团团”离世，岛内民众和许多政治人物都相当不...


In [ ]:
#we can consider not deleting the titles. This will be up to the ICI project group to decide
#if you wish to preserve the titles. replace the corresponding block of code below with this one 

#testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True)#.replace(title_pattern_1,'',regex=True)
#testing_pd['text'] = testing_pd['text'].re place(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
##testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)
#________________________________________________________________________________________________________________________

testing_pd = testing_pd[~testing_pd['text'].str.contains('a href', regex=True)]


import re
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
http_pattern = "http\S+"
image_pattern = "<img\s[^>]*?src\s*=\s*['\"]([^'\"]*?)['\"][^>]*?>" 
#title_pattern = "[【|《*】|》]"
title_pattern_1 = "(?=【)(.*)(?<=】)"
title_pattern_2 = "(?=《)(.*)(?<=》)"
link_pattern="网页链接"
span_pattern='<span.*?>(.+?)</span>'

#replace with this block. Do some more research because it looks like some titles are still removed, like index 41
testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True).replace(title_pattern_1,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)

In [ ]:
#delete empty rows
testing_pd = testing_pd[testing_pd['text'].str.len()>2]   
testing_pd = testing_pd[['text']]
testing_pd = testing_pd.reset_index(drop=True)
testing_pd

,text
0,声光伴读作者：只抽侠移居台湾的YouTuber拍片，用亲身经历告诉大家：移民台湾撞板多个食饭...
1,粤读博览作者：说书人当地有记者问拜登：「乌克兰现在真的必须割地才能换取和平吗？」拜登如此回应...
2,據台灣中時新聞網16日報道，台民進黨近期諸多爭議引發討論，針對民進黨前主席許信良胞弟、前“立...
3,蔡英文20日宣布由现年91岁的台积电创办人张忠谋担任代表，出席11月18日至19日在泰国举行...
4,#湾湾发电姬# 陈时中的仇恨值为什么这么高？他自己说不知道，我们就来帮他知道。台湾老百姓在防...
...,...
181,#赠台大熊猫“团团”不幸离世# L台海网的微博视频 ​
182,「小英」技窮？ //蔡英文獨孤一味搞政治，台灣民眾仲咁蠢想繼續捱窮咩？// ​
183,Residentes de ambos lados del estrecho de Taiw...
184,大陆赠台大熊猫“团团”离世，岛内民众和许多政治人物都相当不舍。但蔡英文和其他民进党政客几乎都...


## 1-2 Tokenization

In [ ]:
from datasets import Dataset, DatasetDict

weibo_dict = {
              'test' : Dataset.from_pandas(testing_pd)
              }

weibo_ds = DatasetDict(weibo_dict)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/config.json
Model config BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=512)

tokenized_datasets = weibo_ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
full_test_dataset = tokenized_datasets["test"]

# 2. preparing BERT


In [ ]:
from transformers import AutoModelForSequenceClassification

# available chinese models: https://github.com/ymcui/Chinese-BERT-wwm

model = AutoModelForSequenceClassification.from_pretrained(model_classifier, 
                                                           num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--owen198--weibo-wmmbert-2/snapshots/1c8b069ca4f42474887b1be5ddb45d0b190b785d/config.json
Model config BertConfig {
  "_name_or_path": "owen198/weibo-wmmbert-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "f

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    eval_dataset=full_test_dataset,
    compute_metrics=compute_metrics,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

#prediction results


In [ ]:
pred = trainer.predict(test_dataset=full_test_dataset)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 186
  Batch size = 8


In [ ]:
pred

PredictionOutput(predictions=array([[ 1.9090811 , -2.5051749 ],
       [ 1.9857025 , -2.588784  ],
       [ 2.1701317 , -2.7997591 ],
       [-2.6510258 ,  2.6624072 ],
       [ 2.2646058 , -2.9690094 ],
       [ 1.956528  , -2.5573483 ],
       [-2.5978484 ,  2.6313307 ],
       [-1.4003962 ,  1.3518429 ],
       [ 1.8395076 , -2.4241247 ],
       [ 1.7592562 , -2.3186822 ],
       [ 1.8330547 , -2.4161985 ],
       [-1.929766  ,  1.9635148 ],
       [-1.8502088 ,  1.8341212 ],
       [-2.6840293 ,  2.6815257 ],
       [-2.52357   ,  2.5860798 ],
       [ 1.896671  , -2.4911957 ],
       [ 2.0207658 , -2.6264217 ],
       [ 1.5580499 , -1.9605317 ],
       [ 2.2324388 , -2.8974142 ],
       [ 0.26664636,  0.1934537 ],
       [ 1.5016901 , -1.8192816 ],
       [ 1.6726071 , -2.1811743 ],
       [ 1.8822109 , -2.4746735 ],
       [ 1.7704033 , -2.334376  ],
       [ 2.2393398 , -2.9109268 ],
       [ 1.9710037 , -2.5729601 ],
       [ 1.5390621 , -1.9164556 ],
       [ 1.5106229 , -1.84

In [ ]:
# logits, labels = pred.predictions
np.argmax(pred.predictions, axis=-1)

array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0])

In [ ]:
testing_pd['pred'] = np.argmax(pred.predictions, axis=-1)

In [ ]:
#final table
testing_pd

,text,pred
0,声光伴读作者：只抽侠移居台湾的YouTuber拍片，用亲身经历告诉大家：移民台湾撞板多个食饭...,0
1,粤读博览作者：说书人当地有记者问拜登：「乌克兰现在真的必须割地才能换取和平吗？」拜登如此回应...,0
2,據台灣中時新聞網16日報道，台民進黨近期諸多爭議引發討論，針對民進黨前主席許信良胞弟、前“立...,0
3,蔡英文20日宣布由现年91岁的台积电创办人张忠谋担任代表，出席11月18日至19日在泰国举行...,1
4,#湾湾发电姬# 陈时中的仇恨值为什么这么高？他自己说不知道，我们就来帮他知道。台湾老百姓在防...,0
...,...,...
181,#赠台大熊猫“团团”不幸离世# L台海网的微博视频 ​,1
182,「小英」技窮？ //蔡英文獨孤一味搞政治，台灣民眾仲咁蠢想繼續捱窮咩？// ​,0
183,Residentes de ambos lados del estrecho de Taiw...,1
184,大陆赠台大熊猫“团团”离世，岛内民众和许多政治人物都相当不舍。但蔡英文和其他民进党政客几乎都...,0


In [ ]:
count0 = (testing_pd['pred'] == 0).sum()
count1 = (testing_pd['pred'] == 1).sum()
print(count0, count1)

108 78


In [ ]:
def Percentage(num1, num2):
    return num1 / (num1 + num2) * 100
    
print(Percentage(count0, count1))
print(Percentage(count1, count0))

58.06451612903226
41.935483870967744


#**蔡英文_after**

# 1. load and clean data

In [ ]:
#insert new xlsx file here
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/蔡英文_after.xlsx

--2022-12-07 04:20:12--  https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/%E8%94%A1%E8%8B%B1%E6%96%87_after.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65781 (64K) [application/octet-stream]
Saving to: ‘蔡英文_after.xlsx’

蔡英文_after.xlsx   100%[===================>]  64.24K  --.-KB/s    in 0.002s  

2022-12-07 04:20:13 (30.8 MB/s) - ‘蔡英文_after.xlsx’ saved [65781/65781]



In [ ]:
import pandas as pd

In [ ]:
#insert new xlsx file here
testing_pd = pd.read_excel('/content/蔡英文_after.xlsx')

testing_pd.rename(columns={'博文': 'text'},
          inplace=True, errors='raise')

testing_pd = testing_pd[['text']]

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#dirty data
testing_pd

,text
0,【#台湾“九合一”选举#今登场 朱立伦斥民进党“由绿转黑”】台湾的“九合一”选举26日投票，...
1,蔡英文宣布辞去民进党主席职务。 #蔡英文宣布辞去民进党主席职务##蔡英文辞去民进党主席职务...
2,【#蔡英文辞去民进党主席职务#】
3,【台湾地区领导人辞去执政党主席职务】据台湾中央社报道，台湾地区行政长官蔡英文在市政选举失败后...
4,【国台办发言人就台湾地区“九合一”选举结果答记者问】国台办发言人朱凤莲26日晚就台湾地区“九...
...,...
242,【这，就是引狼入室的下场】刚刚结束的台湾地区基层公职人员“九合一”选举结果显示，在已选举产生...
243,#蔡英文宣布辞去民进党主席职务##两岸头条# 【“九合一”：蒋万安当选台北市长 民进党大败 ...
244,“九合一”大溃败，台湾民众为何给民进党和蔡英文颜色看？ L中国台湾网的微博视频 ​
245,【#蔡英文辞去民进党主席职务#】//@河北长安网: 【#蔡英文辞去民进党主席职务#】


In [ ]:
#we can consider not deleting the titles. This will be up to the ICI project group to decide
#if you wish to preserve the titles. replace the corresponding block of code below with this one 

#testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True)#.replace(title_pattern_1,'',regex=True)
#testing_pd['text'] = testing_pd['text'].re place(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
##testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)
#________________________________________________________________________________________________________________________

testing_pd = testing_pd[~testing_pd['text'].str.contains('a href', regex=True)]


import re
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
http_pattern = "http\S+"
image_pattern = "<img\s[^>]*?src\s*=\s*['\"]([^'\"]*?)['\"][^>]*?>" 
#title_pattern = "[【|《*】|》]"
title_pattern_1 = "(?=【)(.*)(?<=】)"
title_pattern_2 = "(?=《)(.*)(?<=》)"
link_pattern="网页链接"
span_pattern='<span.*?>(.+?)</span>'

#replace with this block. Do some more research because it looks like some titles are still removed, like index 41
testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True).replace(title_pattern_1,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)

In [ ]:
#delete empty rows
testing_pd = testing_pd[testing_pd['text'].str.len()>2]   
testing_pd = testing_pd[['text']]
testing_pd = testing_pd.reset_index(drop=True)
testing_pd

,text
0,台湾的“九合一”选举26日投票，蓝绿阵营25日尽最后努力拉票。台湾地区领导人蔡英文到多个县市...
1,蔡英文宣布辞去民进党主席职务。 #蔡英文宣布辞去民进党主席职务##蔡英文辞去民进党主席职务...
2,据台湾中央社报道，台湾地区行政长官蔡英文在市政选举失败后于周六辞去执政的民进党主席一职。O台...
3,国台办发言人朱凤莲26日晚就台湾地区“九合一”选举结果应询表示，我们注意到这次选举的结果。这...
4,#蔡英文辞去民进党主席职务#据新华社消息：11月26日晚间，蔡英文宣布辞去民进党主席职务，以...
...,...
236,台湾领导人蔡英文在民进党地方选举失败后辞去民进党主席一职。莫斯科国立国际关系学院国际研究所高...
237,刚刚结束的台湾地区基层公职人员“九合一”选举结果显示，在已选举产生的21个县市长中，民进党仅...
238,#蔡英文宣布辞去民进党主席职务##两岸头条# 台湾地区“九合一”选举民进党大败，蔡英文辞去党...
239,“九合一”大溃败，台湾民众为何给民进党和蔡英文颜色看？ L中国台湾网的微博视频 ​


## 1-2 Tokenization

In [ ]:
from datasets import Dataset, DatasetDict

weibo_dict = {
              'test' : Dataset.from_pandas(testing_pd)
              }

weibo_ds = DatasetDict(weibo_dict)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/config.json
Model config BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=512)

tokenized_datasets = weibo_ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
full_test_dataset = tokenized_datasets["test"]

# 2. preparing BERT


In [ ]:
from transformers import AutoModelForSequenceClassification

# available chinese models: https://github.com/ymcui/Chinese-BERT-wwm

model = AutoModelForSequenceClassification.from_pretrained(model_classifier, 
                                                           num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--owen198--weibo-wmmbert-2/snapshots/1c8b069ca4f42474887b1be5ddb45d0b190b785d/config.json
Model config BertConfig {
  "_name_or_path": "owen198/weibo-wmmbert-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "f

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    eval_dataset=full_test_dataset,
    compute_metrics=compute_metrics,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

#prediction results


In [ ]:
pred = trainer.predict(test_dataset=full_test_dataset)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 241
  Batch size = 8


In [ ]:
pred

PredictionOutput(predictions=array([[ 1.7683256 , -2.3314886 ],
       [ 1.5304408 , -1.8928716 ],
       [ 2.4166534 , -3.5177445 ],
       [-2.6795607 ,  2.678932  ],
       [-0.29623127,  0.7383587 ],
       [ 2.10613   , -2.720769  ],
       [-2.58593   ,  2.6242545 ],
       [-2.6270993 ,  2.6485038 ],
       [-2.6270993 ,  2.6485038 ],
       [-1.7726817 ,  1.7107605 ],
       [ 2.0333538 , -2.6400037 ],
       [ 1.7803729 , -2.348051  ],
       [ 1.7297964 , -2.2753227 ],
       [-1.9376376 ,  1.9757715 ],
       [-1.9376376 ,  1.9757715 ],
       [ 1.7297964 , -2.2753227 ],
       [ 2.10613   , -2.720769  ],
       [-1.9972773 ,  2.0650697 ],
       [ 1.6435195 , -2.1281903 ],
       [-2.0650373 ,  2.1607044 ],
       [ 1.7352328 , -2.283554  ],
       [ 1.7806966 , -2.3485274 ],
       [ 1.6678096 , -2.1727724 ],
       [ 2.0986998 , -2.712237  ],
       [ 2.2963448 , -3.074125  ],
       [ 2.10613   , -2.720769  ],
       [ 1.5609505 , -1.9663756 ],
       [-1.3432915 ,  1.31

In [ ]:
# logits, labels = pred.predictions
np.argmax(pred.predictions, axis=-1)

array([0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1])

In [ ]:
testing_pd['pred'] = np.argmax(pred.predictions, axis=-1)

In [ ]:
#final table
testing_pd

,text,pred
0,台湾的“九合一”选举26日投票，蓝绿阵营25日尽最后努力拉票。台湾地区领导人蔡英文到多个县市...,0
1,蔡英文宣布辞去民进党主席职务。 #蔡英文宣布辞去民进党主席职务##蔡英文辞去民进党主席职务...,0
2,据台湾中央社报道，台湾地区行政长官蔡英文在市政选举失败后于周六辞去执政的民进党主席一职。O台...,0
3,国台办发言人朱凤莲26日晚就台湾地区“九合一”选举结果应询表示，我们注意到这次选举的结果。这...,1
4,#蔡英文辞去民进党主席职务#据新华社消息：11月26日晚间，蔡英文宣布辞去民进党主席职务，以...,1
...,...,...
236,台湾领导人蔡英文在民进党地方选举失败后辞去民进党主席一职。莫斯科国立国际关系学院国际研究所高...,1
237,刚刚结束的台湾地区基层公职人员“九合一”选举结果显示，在已选举产生的21个县市长中，民进党仅...,1
238,#蔡英文宣布辞去民进党主席职务##两岸头条# 台湾地区“九合一”选举民进党大败，蔡英文辞去党...,0
239,“九合一”大溃败，台湾民众为何给民进党和蔡英文颜色看？ L中国台湾网的微博视频 ​,0


In [ ]:
count0 = (testing_pd['pred'] == 0).sum()
count1 = (testing_pd['pred'] == 1).sum()
print(count0, count1)

190 51


In [ ]:
def Percentage(num1, num2):
    return num1 / (num1 + num2) * 100
    
print(Percentage(count0, count1))
print(Percentage(count1, count0))

78.83817427385893
21.16182572614108


#**Chiang before**

# 1. load and clean data

In [ ]:
#insert new xlsx file here
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/Chiang_before.xlsx

--2022-12-07 04:30:33--  https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/Chiang_before.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209228 (204K) [application/octet-stream]
Saving to: ‘Chiang_before.xlsx’

Chiang_before.xlsx  100%[===================>] 204.32K  --.-KB/s    in 0.003s  

2022-12-07 04:30:33 (73.8 MB/s) - ‘Chiang_before.xlsx’ saved [209228/209228]



In [ ]:
import pandas as pd

In [ ]:
#insert new xlsx file here
testing_pd = pd.read_excel('/content/Chiang_before.xlsx')

testing_pd.rename(columns={'博文': 'text'},
          inplace=True, errors='raise')

testing_pd = testing_pd[['text']]

In [ ]:
#dirty data
testing_pd

,text
0,【不只徐巧芯被泼脏水！绿侧翼抹黑高虹安手段曝光 惊人4字现形】9日绿营侧翼流传一份伪造文件，...
1,就说七伤拳嘛并且搞了半天好像只有蒋万安一个人辞职，同党和别党立委都不辞，噗嗤。 ​
2,【台北市长民调跌破眼镜！网一看酸爆陈时中：笑死人】距离九合一选举投票日不到20天，中华亚太菁...
3,最后预言一下，本月26日台湾省九合一大选可能出现的选举结果。（此次选举会影响未来两岸关系发展...
4,台北民调！蒋万安29.8%领先，蒋笃定、黄超车，陈哭哭，蔡咋办？ - 今日头条O网页链接 ​
...,...
785,【台北市长选举民调曝光 3人差距小选情白热化】台北市长选战开打，若由中国国民党推出的“立委”...
786,【台北市长选情最新民调曝光 网惊呼：死忠的这么多】拉锯战！TVBS民调中心昨(29日)发布《...
787,喜猪妹才会投陈时台吧，通讯录当然选蒋万安啊。 ​
788,最新民調，蔣萬安褲暫時領先 ​


In [ ]:
#we can consider not deleting the titles. This will be up to the ICI project group to decide
#if you wish to preserve the titles. replace the corresponding block of code below with this one 

#testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True)#.replace(title_pattern_1,'',regex=True)
#testing_pd['text'] = testing_pd['text'].re place(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
##testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)
#________________________________________________________________________________________________________________________

testing_pd = testing_pd[~testing_pd['text'].str.contains('a href', regex=True)]


import re
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
http_pattern = "http\S+"
image_pattern = "<img\s[^>]*?src\s*=\s*['\"]([^'\"]*?)['\"][^>]*?>" 
#title_pattern = "[【|《*】|》]"
title_pattern_1 = "(?=【)(.*)(?<=】)"
title_pattern_2 = "(?=《)(.*)(?<=》)"
link_pattern="网页链接"
span_pattern='<span.*?>(.+?)</span>'

#replace with this block. Do some more research because it looks like some titles are still removed, like index 41
testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True).replace(title_pattern_1,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)

In [ ]:
#delete empty rows
testing_pd = testing_pd[testing_pd['text'].str.len()>2]   
testing_pd = testing_pd[['text']]
testing_pd = testing_pd.reset_index(drop=True)
testing_pd

,text
0,9日绿营侧翼流传一份伪造文件，指称国民党计划找狗仔跟拍县市长候选人行程，重点针对“黑金、贿选...
1,就说七伤拳嘛并且搞了半天好像只有蒋万安一个人辞职，同党和别党立委都不辞，噗嗤。 ​
2,距离九合一选举投票日不到20天，中华亚太菁英交流协会10日公布台北市长选举最新民调，民进党台...
3,最后预言一下，本月26日台湾省九合一大选可能出现的选举结果。（此次选举会影响未来两岸关系发展...
4,台北民调！蒋万安29.8%领先，蒋笃定、黄超车，陈哭哭，蔡咋办？ - 今日头条O ​
...,...
785,台北市长选战开打，若由中国国民党推出的“立委”蒋万安、民进党征召的前“卫福部长”陈时中及台北...
786,拉锯战！TVBS民调中心昨(29日)发布报告，国民党参选人蒋万安支持度微降至36%，刚辞职的...
787,喜猪妹才会投陈时台吧，通讯录当然选蒋万安啊。 ​
788,最新民調，蔣萬安褲暫時領先 ​


## 1-2 Tokenization

In [ ]:
from datasets import Dataset, DatasetDict

weibo_dict = {
              'test' : Dataset.from_pandas(testing_pd)
              }

weibo_ds = DatasetDict(weibo_dict)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/config.json
Model config BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=512)

tokenized_datasets = weibo_ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
full_test_dataset = tokenized_datasets["test"]

# 2. preparing BERT


In [ ]:
from transformers import AutoModelForSequenceClassification

# available chinese models: https://github.com/ymcui/Chinese-BERT-wwm

model = AutoModelForSequenceClassification.from_pretrained(model_classifier, 
                                                           num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--owen198--weibo-wmmbert-2/snapshots/1c8b069ca4f42474887b1be5ddb45d0b190b785d/config.json
Model config BertConfig {
  "_name_or_path": "owen198/weibo-wmmbert-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "f

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    eval_dataset=full_test_dataset,
    compute_metrics=compute_metrics,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

#prediction results


In [ ]:
pred = trainer.predict(test_dataset=full_test_dataset)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 790
  Batch size = 8


In [ ]:
pred

PredictionOutput(predictions=array([[ 2.277855 , -3.0073154],
       [ 2.403945 , -3.4819767],
       [-2.0893133,  2.1935632],
       ...,
       [ 1.6261932, -2.0954049],
       [-2.5244975,  2.58667  ],
       [-2.4184296,  2.515716 ]], dtype=float32), label_ids=None, metrics={'test_runtime': 27.4326, 'test_samples_per_second': 28.798, 'test_steps_per_second': 3.609})

In [ ]:
# logits, labels = pred.predictions
np.argmax(pred.predictions, axis=-1)

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
testing_pd['pred'] = np.argmax(pred.predictions, axis=-1)

In [ ]:
#final table
testing_pd

,text,pred
0,9日绿营侧翼流传一份伪造文件，指称国民党计划找狗仔跟拍县市长候选人行程，重点针对“黑金、贿选...,0
1,就说七伤拳嘛并且搞了半天好像只有蒋万安一个人辞职，同党和别党立委都不辞，噗嗤。 ​,0
2,距离九合一选举投票日不到20天，中华亚太菁英交流协会10日公布台北市长选举最新民调，民进党台...,1
3,最后预言一下，本月26日台湾省九合一大选可能出现的选举结果。（此次选举会影响未来两岸关系发展...,1
4,台北民调！蒋万安29.8%领先，蒋笃定、黄超车，陈哭哭，蔡咋办？ - 今日头条O ​,0
...,...,...
785,台北市长选战开打，若由中国国民党推出的“立委”蒋万安、民进党征召的前“卫福部长”陈时中及台北...,1
786,拉锯战！TVBS民调中心昨(29日)发布报告，国民党参选人蒋万安支持度微降至36%，刚辞职的...,1
787,喜猪妹才会投陈时台吧，通讯录当然选蒋万安啊。 ​,0
788,最新民調，蔣萬安褲暫時領先 ​,1


In [ ]:
count0 = (testing_pd['pred'] == 0).sum()
count1 = (testing_pd['pred'] == 1).sum()
print(count0, count1)

522 268


In [ ]:
def Percentage(num1, num2):
    return num1 / (num1 + num2) * 100
    
print(Percentage(count0, count1))
print(Percentage(count1, count0))

66.07594936708861
33.92405063291139


#**Chiang_after**

# 1. load and clean data

In [ ]:
#insert new xlsx file here
! wget -nc https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/Chiang_after.xlsx

--2022-12-07 04:31:16--  https://raw.githubusercontent.com/advapplab/sentiment_analysis_election_2022/main/data/challange/Chiang_after.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92178 (90K) [application/octet-stream]
Saving to: ‘Chiang_after.xlsx’

Chiang_after.xlsx   100%[===================>]  90.02K  --.-KB/s    in 0.001s  

2022-12-07 04:31:16 (69.2 MB/s) - ‘Chiang_after.xlsx’ saved [92178/92178]



In [ ]:
import pandas as pd

In [ ]:
#insert new xlsx file here
testing_pd = pd.read_excel('/content/Chiang_after.xlsx')

testing_pd.rename(columns={'博文': 'text'},
          inplace=True, errors='raise')

testing_pd = testing_pd[['text']]

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#dirty data
testing_pd

,text
0,预祝帅气的蒋公曾孙参选台北市长成功。是参选，竞选，是台北人民一张张选票出来的那种。5:/ Z...
1,蒋万安是我看过最认真，最努力的台北市长参选人，他值得赢 ​
2,还是坚持我月初的预测中国国民党：14民主进步党： 6台湾民众党：1无党籍：1台北 蒋万安新北...
3,蔣萬安，單選臺北市市長加油#蒋万安# L高兴15014的微博视频 ​
4,《蒋万安将参选台北市市长，表态将直面蒋家功与过》蒋万安将参选台北市市长，表态将直面蒋家功与过...
...,...
349,【蒋万安公布小内阁用人标准 ：柯市府优秀首长可留任】台北市长当选人蒋万安小内阁备受关注。蒋万...
350,【立委补选“今天内确认人选” 蒋万安：一定要守住这一席】国民党台北市长当选人蒋万安选前辞去立...
351,【柯蒋会来了 蒋万安12／5 访柯市府、林奕华任交接窗口】九合一大选落幕，台北市政府将改朝换...
352,蒋万安赢得中国台湾省台北市市长选选举 ​


In [ ]:
#we can consider not deleting the titles. This will be up to the ICI project group to decide
#if you wish to preserve the titles. replace the corresponding block of code below with this one 

#testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
#testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True)#.replace(title_pattern_1,'',regex=True)
#testing_pd['text'] = testing_pd['text'].re place(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
##testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)
#________________________________________________________________________________________________________________________

testing_pd = testing_pd[~testing_pd['text'].str.contains('a href', regex=True)]


import re
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
http_pattern = "http\S+"
image_pattern = "<img\s[^>]*?src\s*=\s*['\"]([^'\"]*?)['\"][^>]*?>" 
#title_pattern = "[【|《*】|》]"
title_pattern_1 = "(?=【)(.*)(?<=】)"
title_pattern_2 = "(?=《)(.*)(?<=》)"
link_pattern="网页链接"
span_pattern='<span.*?>(.+?)</span>'

#replace with this block. Do some more research because it looks like some titles are still removed, like index 41
testing_pd['text'] = testing_pd['text'].replace(span_pattern,'',regex=True).replace(span_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(http_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(image_pattern,'',regex=True).replace(title_pattern_1,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(link_pattern,'',regex=True).replace(html_pattern,'',regex=True)
testing_pd['text'] = testing_pd['text'].replace(title_pattern_2,'',regex=True)

In [ ]:
#delete empty rows
testing_pd = testing_pd[testing_pd['text'].str.len()>2]   
testing_pd = testing_pd[['text']]
testing_pd = testing_pd.reset_index(drop=True)
testing_pd

,text
0,预祝帅气的蒋公曾孙参选台北市长成功。是参选，竞选，是台北人民一张张选票出来的那种。5:/ Z...
1,蒋万安是我看过最认真，最努力的台北市长参选人，他值得赢 ​
2,还是坚持我月初的预测中国国民党：14民主进步党： 6台湾民众党：1无党籍：1台北 蒋万安新北...
3,蔣萬安，單選臺北市市長加油#蒋万安# L高兴15014的微博视频 ​
4,蒋万安将参选台北市市长，表态将直面蒋家功与过O ​
...,...
349,台北市长当选人蒋万安小内阁备受关注。蒋万安今天表示，他的用人标准非常清楚，就是专业、品格、用...
350,国民党台北市长当选人蒋万安选前辞去立委一职，留下北市第3选举区（中山、北松山）立委悬缺，国民...
351,九合一大选落幕，台北市政府将改朝换代，国民党台北市长当选人蒋万安今天上午表示，下周一早上10...
352,蒋万安赢得中国台湾省台北市市长选选举 ​


## 1-2 Tokenization

In [ ]:
from datasets import Dataset, DatasetDict

weibo_dict = {
              'test' : Dataset.from_pandas(testing_pd)
              }

weibo_ds = DatasetDict(weibo_dict)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--hfl--chinese-bert-wwm/snapshots/ab0aa81da273504efc8540aa4d0bbaa3016a1bb5/config.json
Model config BertConfig {
  "_name_or_path": "hfl/chinese-bert-wwm",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voca

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=512)

tokenized_datasets = weibo_ds.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
full_test_dataset = tokenized_datasets["test"]

# 2. preparing BERT


In [ ]:
from transformers import AutoModelForSequenceClassification

# available chinese models: https://github.com/ymcui/Chinese-BERT-wwm

model = AutoModelForSequenceClassification.from_pretrained(model_classifier, 
                                                           num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--owen198--weibo-wmmbert-2/snapshots/1c8b069ca4f42474887b1be5ddb45d0b190b785d/config.json
Model config BertConfig {
  "_name_or_path": "owen198/weibo-wmmbert-2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "f

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, 
    eval_dataset=full_test_dataset,
    compute_metrics=compute_metrics,
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import numpy as np

#prediction results


In [ ]:
pred = trainer.predict(test_dataset=full_test_dataset)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 354
  Batch size = 8


In [ ]:
pred

PredictionOutput(predictions=array([[-2.3605614 ,  2.4727662 ],
       [-2.691224  ,  2.6857    ],
       [-2.352408  ,  2.466415  ],
       [-2.5702162 ,  2.6148238 ],
       [-2.3860025 ,  2.4920824 ],
       [-2.7356567 ,  2.7118466 ],
       [-1.839245  ,  1.8159657 ],
       [-2.1451442 ,  2.2649488 ],
       [ 2.3438063 , -3.2862785 ],
       [-2.149518  ,  2.2702408 ],
       [-2.3962023 ,  2.4996266 ],
       [-2.6880112 ,  2.6838372 ],
       [-2.666071  ,  2.6711206 ],
       [-2.4800017 ,  2.5579655 ],
       [-1.6873093 ,  1.5985425 ],
       [ 2.2020204 , -2.8455532 ],
       [-1.2660291 ,  1.2671958 ],
       [-2.441676  ,  2.5320437 ],
       [ 1.6299478 , -2.1025875 ],
       [-2.5619724 ,  2.609833  ],
       [ 1.4803815 , -1.7593954 ],
       [ 1.4691107 , -1.7195208 ],
       [-1.9195589 ,  1.9474354 ],
       [-2.5132458 ,  2.5795383 ],
       [-0.42084095,  0.69213146],
       [-2.4747612 ,  2.554497  ],
       [-1.6239815 ,  1.5312512 ],
       [ 1.9309655 , -2.52

In [ ]:
# logits, labels = pred.predictions
np.argmax(pred.predictions, axis=-1)

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,

In [ ]:
testing_pd['pred'] = np.argmax(pred.predictions, axis=-1)

In [ ]:
#final table
testing_pd

,text,pred
0,预祝帅气的蒋公曾孙参选台北市长成功。是参选，竞选，是台北人民一张张选票出来的那种。5:/ Z...,1
1,蒋万安是我看过最认真，最努力的台北市长参选人，他值得赢 ​,1
2,还是坚持我月初的预测中国国民党：14民主进步党： 6台湾民众党：1无党籍：1台北 蒋万安新北...,1
3,蔣萬安，單選臺北市市長加油#蒋万安# L高兴15014的微博视频 ​,1
4,蒋万安将参选台北市市长，表态将直面蒋家功与过O ​,1
...,...,...
349,台北市长当选人蒋万安小内阁备受关注。蒋万安今天表示，他的用人标准非常清楚，就是专业、品格、用...,1
350,国民党台北市长当选人蒋万安选前辞去立委一职，留下北市第3选举区（中山、北松山）立委悬缺，国民...,1
351,九合一大选落幕，台北市政府将改朝换代，国民党台北市长当选人蒋万安今天上午表示，下周一早上10...,1
352,蒋万安赢得中国台湾省台北市市长选选举 ​,1


In [ ]:
count0 = (testing_pd['pred'] == 0).sum()
count1 = (testing_pd['pred'] == 1).sum()
print(count0, count1)

105 249


In [ ]:
def Percentage(num1, num2):
    return num1 / (num1 + num2) * 100
    
print(Percentage(count0, count1))
print(Percentage(count1, count0))

29.66101694915254
70.33898305084746
